In [15]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import numpy as np

In [16]:
ENTITY_TYPE = "skill_card"

MODEL_PATH = f'{ENTITY_TYPE}_model.pt'
IMAGE_DIR = f"gk-img/docs/{ENTITY_TYPE}s/icons"
OUTPUT_PATH = f"{ENTITY_TYPE}_embeddings.json"
EMBEDDING_DIM = 128

In [17]:
class SmallEmbeddingNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64 → 32
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32 → 16
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),  # → 1×1 output
        )
        self.embedding = nn.Linear(128, embedding_dim)

    def forward(self, x):
        x = self.conv(x).view(x.size(0), -1)
        emb = self.embedding(x)
        emb = F.normalize(emb, p=2, dim=1)
        return emb

In [18]:
model = SmallEmbeddingNet(embedding_dim=EMBEDDING_DIM)
model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"), strict=False)
model.eval()

SmallEmbeddingNet(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (embedding): Linear(in_features=128, out_features=128, bias=True)
)

In [19]:
transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ]
)

In [20]:
embeddings = {}

with torch.no_grad():
    for filename in sorted(os.listdir(IMAGE_DIR)):
        card_id = os.path.splitext(filename)[0]
        path = os.path.join(IMAGE_DIR, filename)
        image = Image.open(path).convert("RGB")
        tensor = transform(image).unsqueeze(0)
        embedding = model(tensor).squeeze().tolist()
        embeddings[card_id] = embedding

In [21]:
embeddings[card_id] = [
    float(x) for x in np.array(embedding, dtype=np.float16)
]  # quantize to float16
with open(OUTPUT_PATH, "w") as f:
    json.dump(embeddings, f)

print(f"✅ Saved {len(embeddings)} card embeddings to {OUTPUT_PATH}")

✅ Saved 2570 card embeddings to skill_card_embeddings.json
